__Author: Manu Jayadharan, University of Pittsburgh, 2020__

# Using fluidlearn to solve diffusion equation

Equation to solve: $u_t-\Delta u -f  = 0$
over domain $\Omega$ from time T_initial to T_final.

For demonstration purposes we take $f=sin(x_1 + x_2) + tsin(x_1 + x_2)$ and $\Omega = [-2,2]\times [0,1]$ and the time interval to be $[0,1]$, so we can compare the results with the actual solution $u=tsin(x_1 + x_2)$.

In [1]:
#Import fluidlearn package and classes
import fluidlearn
from fluidlearn import dataprocess

### Defining the domain and time interval for which the PDE needs to be solved.
This matters only for generating collocation points and if the user is feeding their own collocation points,
they can skip this step.

In [2]:
#domain range
X_1_domain = [-2, 2]
X_2_domain = [0, 1]
#time range
T_initial = 0
T_final = 1
T_domain = [T_initial, T_final]

#domain of the problem
domain_bounds = [X_1_domain, X_2_domain, T_domain]

### Loading data from a csv file
- We use the manufactured data with $u=tsin(x_1 + x_2)$ saved in a csv file.
- Data is saved in the format: ($x_1 , x_2, t, u(x_1, x_2, t)$) as four columns.
- You could load either preprocess your data to be in this format or load your data
  from a csv file with similar format.

In [3]:
path_to_data = "data_manufactured/t_sin_x_plus_y.csv"
X_data, Y_data = dataprocess.imp_from_csv(path_to_csv_file=path_to_data,
                                           x_y_combined=True, y_dim=1)

### Defining the rhs function $f=sin(x_1 + x_2) + tsin(x_1 + x_2)$ of the PDE.
We use tensorflow.sin function instead of python functions, we could used numpy.sin as well.

In [4]:
def rhs_function (args, time_dep=True):
        import tensorflow as tf
        if time_dep:
            space_inputs = args[:-1]
            time_inputs = args[-1]
        else:
            space_inputs = args
        
        return tf.sin(space_inputs[0]+space_inputs[1]) + 2*time_inputs*tf.sin(space_inputs[0]+space_inputs[1])

### Defining the model architecture

In [5]:
model_type = 'forward'
space_dim = 2 #dimension of Omega
time_depedent_problem = True
n_hid_lay=3 #numberof hidden layers in the neural network
n_hid_nrn=20 #number of neurons in each hidden layer
act_func='tanh' #activation function used for hidden layers:  could be elu, relu, sigmoid
loss_list='mse' #type of error function used for cost functin, we use mean squared error.
optimizer='adam' #type of optimizer for cost function minimization
dom_bounds=domain_bounds #domain bounds where collocation points has to be generated

distribution = 'uniform' #type of distribution used for generating the pde collocation points.
number_of_collocation_points = 5000

batch_size = 32 #batch size for stochastic batch gradient type optimization
num_epochs = 10 #number of epochs used for trainng  

### Defining the fluidlearn solver 

In [41]:
diffusion_model = fluidlearn.Solver()

In [42]:
diffusion_model(model_type=model_type,
            space_dim=space_dim,
            time_dep=time_depedent_problem,
            output_dim=1,
            n_hid_lay=n_hid_lay,
            n_hid_nrn=n_hid_lay,
            act_func=act_func,
            rhs_func=rhs_function,
            loss_list=loss_list,
            optimizer=optimizer,
            dom_bounds=dom_bounds,
            load_model=False,
            model_path=None,)

### Fitting the model

In [44]:
diffusion_model.fit(
    x=X_data,
    y=Y_data,
    colloc_points=number_of_collocation_points,
    dist=distribution,
    batch_size=batch_size,
    epochs=num_epochs,
)

Epoch 1/10
219/219 [==============================] - 0s 2ms/step - loss: 0.2146 - output_1_loss: 0.0297 - output_2_loss: 0.1849
Epoch 2/10
219/219 [==============================] - 0s 2ms/step - loss: 0.1877 - output_1_loss: 0.0355 - output_2_loss: 0.1521
Epoch 3/10
219/219 [==============================] - 0s 2ms/step - loss: 0.1598 - output_1_loss: 0.0388 - output_2_loss: 0.1210
Epoch 4/10
219/219 [==============================] - 0s 2ms/step - loss: 0.1298 - output_1_loss: 0.0367 - output_2_loss: 0.0931
Epoch 5/10
219/219 [==============================] - 0s 1ms/step - loss: 0.0966 - output_1_loss: 0.0304 - output_2_loss: 0.0662
Epoch 6/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0709 - output_1_loss: 0.0228 - output_2_loss: 0.0480
Epoch 7/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0572 - output_1_loss: 0.0194 - output_2_loss: 0.0378
Epoch 8/10
219/219 [==============================] - 0s 2ms/step - loss: 0.0490 - output_1_loss:

### Resuming Training  the model again for 50 more epochs

In [45]:
diffusion_model.fit(
    x=X_data,
    y=Y_data,
    colloc_points=number_of_collocation_points,
    dist=distribution,
    batch_size=batch_size,
    epochs=50,
)

Epoch 1/50
219/219 [==============================] - 0s 2ms/step - loss: 0.0367 - output_1_loss: 0.0160 - output_2_loss: 0.0207
Epoch 2/50
219/219 [==============================] - 0s 2ms/step - loss: 0.0343 - output_1_loss: 0.0150 - output_2_loss: 0.0193
Epoch 3/50
219/219 [==============================] - 0s 2ms/step - loss: 0.0320 - output_1_loss: 0.0145 - output_2_loss: 0.0176
Epoch 4/50
219/219 [==============================] - 0s 2ms/step - loss: 0.0304 - output_1_loss: 0.0138 - output_2_loss: 0.0166
Epoch 5/50
219/219 [==============================] - 0s 1ms/step - loss: 0.0286 - output_1_loss: 0.0129 - output_2_loss: 0.0157
Epoch 6/50
219/219 [==============================] - 0s 2ms/step - loss: 0.0270 - output_1_loss: 0.0122 - output_2_loss: 0.0148
Epoch 7/50
219/219 [==============================] - 0s 2ms/step - loss: 0.0254 - output_1_loss: 0.0113 - output_2_loss: 0.0141
Epoch 8/50
219/219 [==============================] - 0s 2ms/step - loss: 0.0242 - output_1_loss:

### Saving the model to a specified location.

In [48]:
path_to_save_model = "saved_model/model_name"
diffusion_model.save_model(path_to_save_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/model_name\assets


### Loading the saved model 

In [6]:
path_to_load_model = "saved_model/model_name"

In [7]:
loaded_diffusion_model = fluidlearn.Solver()

In [8]:
loaded_diffusion_model(space_dim=2,
    time_dep=True,
    load_model=True,
    model_path=path_to_load_model)

### Predicting using loaded model

In [9]:
y_predicted = loaded_diffusion_model.predict(X_data)

In [10]:
y_predicted

array([[-0.10157388],
       [-0.4190994 ],
       [-0.7965628 ],
       ...,
       [-0.10375804],
       [ 0.05802408],
       [-0.00470909]], dtype=float32)